In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 1. Load video
video_path = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/sample video.mp4'
cap = cv2.VideoCapture(video_path)

# 2. Frame Extraction
def extract_frames(video_path, output_folder, frame_interval=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_count = 0
    success = True

    while success:
        success, frame = cap.read()
        if success and frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()
    print(f"Frames extracted and saved to {output_folder}")

# Example usage for frame extraction
video_path = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/sample video.mp4'
output_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/frames'
extract_frames(video_path, output_folder, frame_interval=1)



Frames extracted and saved to /Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/frames


In [5]:
# 3. Spatio-Temporal segmentation
# edge detection
def detect_edges(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.jpg')])

    for i, frame_file in enumerate(frame_files):
        frame_path = os.path.join(input_folder, frame_file)
        frame = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale

        # 1. Apply Gaussian Blur to reduce noise
        blurred = cv2.GaussianBlur(frame, (5, 5), 0)

        # 2. Canny Edge Detection
        edges = cv2.Canny(blurred, 50, 150)  # Adjust thresholds as needed

        # 3. Save Edge-Detected Frame
        output_path = os.path.join(output_folder, f"edges_{frame_file}")
        cv2.imwrite(output_path, edges)
# Example usage
video_path = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/sample video.mp4'
frames_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/frames'
edges_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/edge_frames'  
extract_frames(video_path, frames_folder, frame_interval=1)
detect_edges(frames_folder, edges_folder)
#Track segmented objects using Optical Flow
def track_objects(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.jpg')])
    previous_gray = None  # Initialize as None

    for i, frame_file in enumerate(frame_files):
        frame_path = os.path.join(input_folder, frame_file)
        current_gray = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)  # Directly load as grayscale
        
        if current_gray is None:
            print(f"Error reading frame {frame_file}")
            continue

        if previous_gray is not None:
            # Calculate Optical Flow using Farneback method
            flow = cv2.calcOpticalFlowFarneback(previous_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

            # Visualize Optical Flow (draw flow lines)
            h, w = current_gray.shape[:2]
            y, x = np.mgrid[0:h:10, 0:w:10].reshape(2, -1).astype(int)
            fx, fy = flow[y, x].T
            lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
            lines = np.int32(lines + 0.5)

            vis = cv2.cvtColor(current_gray, cv2.COLOR_GRAY2BGR)  # Convert back to color for visualization
            cv2.polylines(vis, lines, 0, (0, 255, 0))  # Draw green flow lines

            # Save the tracked frame
            output_path = os.path.join(output_folder, f"tracked_{frame_file}")
            cv2.imwrite(output_path, vis)

        # Update previous_gray for the next iteration
        previous_gray = current_gray
# Example usage
video_path = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/sample video.mp4'
frames_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/frames'
edges_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/edge_frames'
tracked_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/tracked_frames'
# ... (call extract_frames and detect_edges) ...
extract_frames(video_path, frames_folder, frame_interval=1)
detect_edges(frames_folder, edges_folder)
track_objects(edges_folder, tracked_folder)

Frames extracted and saved to /Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/frames
Frames extracted and saved to /Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/frames


In [7]:
# Foreground vs. Background Segmentation
def segment_foreground(video_path, output_folder, background_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if not os.path.exists(background_folder):
        os.makedirs(background_folder)

    cap = cv2.VideoCapture(video_path)
    fgbg = cv2.createBackgroundSubtractorMOG2()  # Background subtractor

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Apply the background subtractor
        fgmask = fgbg.apply(frame)
        
        # Save the foreground mask
        foreground_output_path = os.path.join(output_folder, f"foreground_{frame_count}.jpg")
        cv2.imwrite(foreground_output_path, fgmask)

        # Create the background image
        background = fgbg.getBackgroundImage()
        if background is not None:
            background_output_path = os.path.join(background_folder, f"background_{frame_count}.jpg")
            cv2.imwrite(background_output_path, background)
        frame_count += 1 
    
    cap.release()
    cv2.destroyAllWindows()

# Example usage
foreground_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/foreground_frames'
background_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/background_frames'
segment_foreground(video_path, foreground_folder, background_folder)


In [11]:
# 4. Scene Detection for Hard and Soft Cuts
# Function to calculate absolute pixel difference for hard cuts
def detect_hard_cuts(frames, threshold=1000000):  
    cuts = []
    for i in range(1, len(frames)):
        diff = cv2.absdiff(frames[i], frames[i-1])
        non_zero_count = np.count_nonzero(diff)
        if non_zero_count > threshold:
            cuts.append(i)
    return cuts

# Function to calculate histogram difference for soft cuts
# Hard cut frames are excluded from soft cut detection
def detect_soft_cuts(frames, hard_cuts, threshold=0.998):
    cuts = []
    for i in range(1, len(frames)):
        # Skip if the frame is a hard cut
        if i in hard_cuts:
            continue
        hist1 = cv2.calcHist([frames[i-1]], [0], None, [256], [0, 256])
        hist2 = cv2.calcHist([frames[i]], [0], None, [256], [0, 256])
        hist_diff = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
        if hist_diff < threshold:
            cuts.append(i)
    return cuts

#5. Function to mark detected cuts in the frames and
#6.Result visualization
def mark_cuts(frames, hard_cuts, soft_cuts):
    marked_frames = []
    for i, frame in enumerate(frames):
        if i in hard_cuts:
            cv2.putText(frame, "Hard Cut", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            frame = cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 5)
        elif i in soft_cuts:
            cv2.putText(frame, "Soft Cut", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            frame = cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 5)
        marked_frames.append(frame)
    return marked_frames
# Path to the folder containing segmented frames (use your existing frames folder)
segmented_frames_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/foreground_frames'
# Path to the folder to save marked frames with scene cuts
marked_frames_folder = '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/marked_frames'
# Create folder to save the marked frames if it doesn't exist
if not os.path.exists(marked_frames_folder):
    os.makedirs(marked_frames_folder)
# Get the list of frame files in the segmented frames folder
frame_files = sorted([f for f in os.listdir(segmented_frames_folder) if f.endswith('.jpg')])
# Load all the frames into a list
frames = []
for frame_file in frame_files:
    frame_path = os.path.join(segmented_frames_folder, frame_file)
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Error reading frame {frame_file}")
        continue
    frames.append(frame)
# Detect hard cuts
hard_cuts = detect_hard_cuts(frames)
# Detect soft cuts, excluding frames with hard cuts
soft_cuts = detect_soft_cuts(frames, hard_cuts)
# Mark the frames where cuts are detected
marked_frames = mark_cuts(frames, hard_cuts, soft_cuts)
# Save the marked frames
for i, frame in enumerate(marked_frames):
    marked_frame_path = os.path.join(marked_frames_folder, frame_files[i])
    cv2.imwrite(marked_frame_path, frame)
print(f"Hard cuts detected at: {hard_cuts}")
print(f"Soft cuts detected at: {soft_cuts}")
print(f"Marked frames saved in '{marked_frames_folder}'")


Hard cuts detected at: [1, 2, 3, 13, 14, 50, 57, 58, 68, 69, 79, 80, 90, 91, 101, 102, 103, 122, 124, 125, 135, 136, 146, 147, 157, 158, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 212, 213, 223, 225, 235, 236, 246, 247, 257, 258, 266, 268, 269, 276, 277, 278, 279, 280, 283, 284, 307, 308, 312, 313, 320, 323, 324, 327, 328, 333, 334, 336, 337, 338, 339, 343, 348, 349, 350, 351, 352, 354]
Soft cuts detected at: [46, 47, 87, 88, 112, 113, 114, 290, 291, 335, 347, 353, 355]
Marked frames saved in '/Users/madha/OneDrive/Desktop/fall sem 2024/image and video analytics/lab/lab 4/marked_frames'
